# In this project I will attempt to answer questions from a data set using a pre-trained BERT model


In [ ]:
import torch
# Google API
import google

# BERT models
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

import speech_recognition as sr

# Google search API
from googlesearch import search
# Wikipedia API
import wikipedia

import re

# Google translate API
from gtts import gTTS

import os 


# BERT QA modelling
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


#### Speach recognition

In [12]:
def ask():
    # obtain audio from the microphone
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Ask a question!")
        audio = r.listen(source)

    # recognize speech using Google Speech Recognition
    try:
        # for testing purposes, we're just using the default API key
        # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
        # instead of `r.recognize_google(audio)`
        question =  r.recognize_google(audio)
        print("Google Speech Recognition thinks you said " + question)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio.\n Would you like to type in your question or try speach again.\nIf you would like to try again press 'y'!")
        In = input()
        if In == y:
            ask()
        else:
            question = In
            return question
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        print('Would you like to type in your question or try speach again.\nIf you would like to try again press y!')
        In = input()
        if In == y:
            ask()
        else:
            question = In
            return question
    return question

#### Google search

In [8]:
import ddg3
r = ddg3.query('what is the temperature')
for i in range(len(r.related)):
    print(r.related[i].text)

print(r)
r.answer

Temperature Category
State functions
SI base quantities
Continuum mechanics
Thermodynamics


In [6]:
from duckduckpy import query
response = query('what is the temperature') # namedtuple is used as a container

response.abstract
response.infobox

''

In [12]:
from duckduckpy import query
response = query('Age of the universe') # namedtuple is used as a container



In [ ]:
for i in range(len(responce.related_topics)):
    print(response.related_topics[i].text)

In [16]:

def goog(question):
    query = question

    Websites = []
    for j in search(query, tld="co.in", num=10, stop=10, pause=2): 
        Websites.append(j)
    return Websites

Websites = goog(question)
print(Websites)

NameError: name 'question' is not defined

#### Wiki search

In [22]:

def wiki(url):
    if len(re.findall('wikipedia.org', url)) >= 1:
        return url.rsplit('/', 1)[-1]
    else:
        return None
        
def wiki_serach(Websites):
    A=[]
    for url in Websites:
        wik = wiki(url)
        if wik != None:
            A.append(wik)
    return A


def Wiki(Websites):
    pages = wiki_serach(Websites)
    text = wikipedia.page(pages[0]).summary
    
    text = text.split(' ')
    Text = ''
    for word in text:
        Text.join(word)
        
    return Text



TypeError: Wiki() missing 1 required positional argument: 'Websites'

#### question answering

In [18]:
#question = "Age of the universe"
    
def bert(text, question): 
    
    answer_text = text
    
    input_ids=tokenizer.encode(question,answer_text)
    # BERT only needs the token IDs, but for the purpose of inspecting the 
    # tokenizer's behavior, let's also get the token strings and display them.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)



    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

     #Run our example through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                     token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text


    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])

    print('Question: ' + question)
    print('Answer: "' + answer + '"')


    answer = re.sub(' ##', '', answer)
    print(answer)
    return answer

#### Answer to speach

In [19]:

def to_speach(answer):
    # The text that you want to convert to audio 
    mytext = answer

    # Language in which you want to convert 
    language = 'en-uk'

    # Passing the text and language to the engine,  
    # here we have marked slow=False. Which tells  
    # the module that the converted audio should  
    # have a high speed 
    myobj = gTTS(text=mytext, lang=language, slow=False) 

    # Saving the converted audio in a mp3 file named 
    # welcome  
    myobj.save("audio.mp3") 


    from pygame import mixer  # Load the popular external library

    mixer.init()
    mixer.music.load("audio.mp3")
    mixer.music.play() 
    mixer.music.stop()
    os.remove("audio.mp3")


In [25]:
def QA():
    question = ask()
    Websites = goog(question)
    print(Websites)
    text = Wiki(Websites)
    answer = bert(text, question)
    to_speach(answer)
    
QA()

Ask a question!
Google Speech Recognition thinks you said did Kennedy become president
['https://en.wikipedia.org/wiki/John_F._Kennedy', 'https://www.whitehouse.gov/about-the-white-house/presidents/john-f-kennedy/', 'https://www.history.com/this-day-in-history/john-f-kennedy-elected-president', 'https://www.jfklibrary.org/learn/about-jfk/life-of-john-f-kennedy', 'https://www.jfklibrary.org/visit-museum/exhibits/past-exhibits/the-making-of-a-president', 'https://www.britannica.com/biography/John-F-Kennedy', 'https://www.biography.com/us-president/john-f-kennedy', 'https://millercenter.org/president/kennedy/life-before-the-presidency', 'https://millercenter.org/president/kennedy/impact-and-legacy', 'https://www.khanacademy.org/humanities/us-history/postwarera/1960s-america/a/john-f-kennedy-as-president']


Token indices sequence length is longer than the specified maximum sequence length for this model (4 > 512). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self